In [1]:
import os
import sys
sys.path.append("../")
from scipy.io import savemat, loadmat
from IPG.src.lossfunction import LogisticLoss
from IPG.src.regularizer import NatOG
from IPG.src.solver import IpgSolver
import IPG.src.utils as utils
from IPG.src.params import *
import yaml
import numpy as np

In [2]:
def main(lam_shrink = 0.1, datasetName = 'a9a', dbDir = '../../GroupFaRSA/db', grp_size = 10, 
    overlap_ratio = 0.2):
    loss = 'logit'
    fileType = fileTypeDict[datasetName]
    print("Working on: {}...".format(datasetName))
    X, y = utils.set_up_xy(datasetName, fileType, dbDir)
    if loss == 'logit':
        f = LogisticLoss(X, y, datasetName)
    p = X.shape[1]
    grp_size = min(p // 2, grp_size)
    generator = utils.GenOverlapGroup(
        p, grp_size=grp_size, overlap_ratio=overlap_ratio)
    groups, starts, ends = generator.get_group()

    lammax_path = f'{dbDir}/lammax-{datasetName}-{grp_size}-{overlap_ratio}.mat'
    if os.path.exists(lammax_path):
        lammax = loadmat(lammax_path)["lammax"][0][0]
        print(f"loading lammax from: {lammax_path}")
    else:
        lammax = utils.lam_max(X, y, starts, ends, loss)
        savemat(lammax_path, {"lammax": lammax})
        print(f"save lammax to: {lammax_path}")

    r = NatOG(penalty=lammax * lam_shrink, groups=groups, weights=None)


    alpha_init = 1.0

    print(f"Lambda use:{lammax * lam_shrink} | overlap ratio:{overlap_ratio} | group size:{grp_size}")
    print("Inexact subprobsolve: schimdt")
    with open('../IPG/src/config.yaml', "r") as stream:
        config = yaml.load(stream, Loader=yaml.SafeLoader)
    config['mainsolver']['exact_pg_computation'] = False
    config['mainsolver']['inexact_pg_computation'] = 'schimdt'
    config['inexactpg']['schimdt']['c'] = 1e3
    solver = IpgSolver(f, r, config)
    info_schimdt = solver.solve(alpha_init=alpha_init, save_ckpt=False)

    print("Inexact subprobsolve: lee")
    with open('../IPG/src/config.yaml', "r") as stream:
        config = yaml.load(stream, Loader=yaml.SafeLoader)
    config['mainsolver']['exact_pg_computation'] = False
    config['mainsolver']['inexact_pg_computation'] = 'lee'
    config['inexactpg']['lee']['gamma'] = 0.5
    solver = IpgSolver(f, r, config)
    info_lee = solver.solve(alpha_init=alpha_init,  save_ckpt=False)

    print("Inexact subprobsolve: yd")
    with open('../IPG/src/config.yaml', "r") as stream:
        config = yaml.load(stream, Loader=yaml.SafeLoader)
    config['mainsolver']['exact_pg_computation'] = False
    config['mainsolver']['inexact_pg_computation'] = 'yd'
    config['inexactpg']['yd']['gamma'] = 0.1    
    solver = IpgSolver(f, r, config)
    info_yd = solver.solve(alpha_init=alpha_init,  save_ckpt=False)
    print(f"Yd-      #nz:{info_yd['nz']}/#nnz:{info_yd['nnz']}\nlee-     #nz:{info_lee['nz']}/#nnz:{info_lee['nnz']}\nschimdt- #nz:{info_schimdt['nz']}/#nnz:{info_schimdt['nnz']}")
    return info_schimdt, info_lee, info_yd

# a9a dataset

In [4]:
info_schimdt, info_lee, info_yd = main(lam_shrink = 0.5, datasetName = 'a9a', dbDir = '../../GroupFaRSA/db')

Working on: a9a...
loading lammax from: ../../GroupFaRSA/db/lammax-a9a-10-0.2.mat
Lambda use:0.06729056227972936 | overlap ratio:0.2 | group size:10
Inexact subprobsolve: schimdt

Exit: Optimal Solution Found

Inexact subprobsolve: lee

Exit: Optimal Solution Found

Inexact subprobsolve: yd

Exit: Optimal Solution Found

Yd-      #nz:14/#nnz:2
lee-     #nz:14/#nnz:2
schimdt- #nz:14/#nnz:2


In [3]:
info_schimdt, info_lee, info_yd = main(lam_shrink = 0.1, datasetName = 'a9a', dbDir = '../../GroupFaRSA/db')

Working on: a9a...
loading lammax from: ../../GroupFaRSA/db/lammax-a9a-10-0.2.mat
Lambda use:0.013458112455945873 | overlap ratio:0.2 | group size:10
Inexact subprobsolve: schimdt

Exit: Optimal Solution Found

Inexact subprobsolve: lee

Exit: Optimal Solution Found

Inexact subprobsolve: yd

Exit: Optimal Solution Found

Yd-      #nz:11/#nnz:5
lee-     #nz:11/#nnz:5
schimdt- #nz:11/#nnz:5


In [6]:
info_schimdt, info_lee, info_yd = main(lam_shrink = 0.01, datasetName = 'a9a', dbDir = '../../GroupFaRSA/db')

Working on: a9a...
loading lammax from: ../../GroupFaRSA/db/lammax-a9a-10-0.2.mat
Lambda use:0.0013458112455945873 | overlap ratio:0.2 | group size:10
Inexact subprobsolve: schimdt

Exit: Optimal Solution Found

Inexact subprobsolve: lee

Exit: Optimal Solution Found

Inexact subprobsolve: yd

Exit: Optimal Solution Found

Yd-      #nz:8/#nnz:8
lee-     #nz:8/#nnz:8
schimdt- #nz:8/#nnz:8


# colon-cancer dataset

In [10]:
info_schimdt, info_lee, info_yd = main(lam_shrink = 0.5, datasetName = 'colon_cancer', dbDir = '../../GroupFaRSA/db', grp_size = 10, overlap_ratio = 0.1)

Working on: colon_cancer...
loading lammax from: ../../GroupFaRSA/db/lammax-colon_cancer-10-0.1.mat
Lambda use:0.08875471937724845 | overlap ratio:0.1 | group size:10
Inexact subprobsolve: schimdt

Exit: Optimal Solution Found

Inexact subprobsolve: lee

Exit: Optimal Solution Found

Inexact subprobsolve: yd

Exit: Optimal Solution Found

Yd-      #nz:217/#nnz:6
lee-     #nz:217/#nnz:6
schimdt- #nz:217/#nnz:6


In [8]:
info_schimdt, info_lee, info_yd = main(lam_shrink = 0.1, datasetName = 'colon_cancer', dbDir = '../../GroupFaRSA/db', grp_size = 10, overlap_ratio = 0.1)

Working on: colon_cancer...
loading lammax from: ../../GroupFaRSA/db/lammax-colon_cancer-10-0.1.mat
Lambda use:0.01775094387544969 | overlap ratio:0.1 | group size:10
Inexact subprobsolve: schimdt

Exit: Optimal Solution Found

Inexact subprobsolve: lee

Exit: Optimal Solution Found

Inexact subprobsolve: yd

Exit: Optimal Solution Found

Yd-      #nz:213/#nnz:10
lee-     #nz:213/#nnz:10
schimdt- #nz:213/#nnz:10


In [11]:
info_schimdt, info_lee, info_yd = main(lam_shrink = 0.01, datasetName = 'colon_cancer', dbDir = '../../GroupFaRSA/db', grp_size = 10, overlap_ratio = 0.1)

Working on: colon_cancer...
loading lammax from: ../../GroupFaRSA/db/lammax-colon_cancer-10-0.1.mat
Lambda use:0.001775094387544969 | overlap ratio:0.1 | group size:10
Inexact subprobsolve: schimdt

Exit: Optimal Solution Found

Inexact subprobsolve: lee

Exit: Optimal Solution Found

Inexact subprobsolve: yd

Exit: Optimal Solution Found

Yd-      #nz:201/#nnz:22
lee-     #nz:201/#nnz:22
schimdt- #nz:201/#nnz:22
